In [ ]:
import numpy as np
import pandas as pd
import param
import hvplot.pandas

In [ ]:
df_fuel = pd.read_csv('./fuel_cons.csv')
df_emissions = pd.read_csv('./emissions.csv')

In [ ]:
names = {'bah':'Bahrain',
         'kuw':'Kuwait',
         'ksa':'Saudi Arabia',
         'omn':'Oman',
         'qat':'Qatar',
         'uae':'UAE'}

df_fuel = df_fuel.replace(names)
df_emissions = df_emissions.replace(names)

In [ ]:
countries = ['Bahrain','Kuwait','Oman','Qatar','Saudi Arabia','UAE']
fuels = ['HFO','diesel','NG','Arablight']
scenarios = ['A','B','C','D','E','F','G','H']

In [ ]:
df_fuel.info()

In [ ]:
df_emissions.info()

In [ ]:
class GENERAL(param.Parameterized):
    country_selector = param.ObjectSelector(default="Saudi Arabia", objects=countries)
    scenario_selector = param.ObjectSelector(default="H", objects=scenarios)
    year_range = param.Range(default=(2015, 2030), bounds=(2015, 2030))
    show_legend = param.Boolean(False, doc="Show the legend")

# FUEL ----------------------------------------------------------------    
    
    def get_data_fuel(self,country):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        #country = self.country_selector
        
        mask = (df_fuel['c'] == country) & (df_fuel['trun'] >= start_date) & (df_fuel['trun'] <= end_date)
        
        _df = df_fuel.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df

    def plot_fuel(self):     
        country = self.country_selector
        _df  = self.get_data_fuel(country)
        mask = (_df['scenario']==self.scenario_selector)               # come back to this
        data = _df.loc[mask].reset_index(drop=True)      
        
        title = self.country_selector + ' ' + self.scenario_selector
        
        fuel_list = data.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot.bar(x='trun',
                               y=fuel_list,
                               value_label='GW',
                               title=title,
                               alpha=0.4,
                               stacked=True,
                               width=300,
                               legend=self.show_legend,
                               shared_axes=False,
                               rot=90)
        return plot
    
# EMISSIONS ---------------------------------------------------------------   

    def get_data_emissions(self,country):
        start_date = self.year_range[0]
        end_date = self.year_range[1]
        
        mask = (df_emissions['c'] == country) & (df_emissions['trun'] >= start_date) & (df_emissions['trun'] <= end_date)
        _df = df_emissions.loc[mask]
        
        #df_pivot = _df.pivot_table(index=['trun','scenario'],columns='c',values='value',fill_value=0)
        #df_pivot = df_pivot.reset_index(drop=False)
        
        return _df
    
    def plot_emissions(self):
        country = self.country_selector
        _df  = self.get_data_emissions(country)
        mask = (_df['scenario']==self.scenario_selector)
        data = _df.loc[mask].reset_index(drop=True)   
        
        title = 'Emissions for ' + country + ' for ' + self.scenario_selector
        
        #c_list = data.drop(['trun','scenario'],axis=1).columns.values.tolist()

        plot = data.hvplot(
            x='trun',
            value_label='million tons',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=False,
            rot=90)* data.hvplot.scatter(
            x='trun',
            title=title,
            line_color='k',
            shared_axes=True)
        return plot

In [ ]:
class EMISSIONS(GENERAL):
    
# EMISSIONS SUBPLOTS
# https://hvplot.holoviz.org/user_guide/Subplots.html

    def plot_emissions_grid(self):
        country = self.country_selector
        _df  = self.get_data_emissions(country)
        
        title = 'Emissions for ' + country
        
        plot = (_df.hvplot(
            x='trun',
            y='value',
            by='scenario',
            subplots=True,
            value_label='million tons',
            alpha=0.7,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90) * _df.hvplot.scatter(
            x='trun',
            y='value',
            by='scenario',
            subplots=True,
            title=title,
            line_color='k',
            shared_axes=True)).cols(4)
        return plot

In [ ]:
class FUEL(GENERAL):
    def plot_fuel_grid(self):    

        country = self.country_selector
        _df  = self.get_data_fuel(country)
        ##mask = (_df['scenario']==self.scenario_selector)               # come back to this
        ##data = _df.loc[mask].reset_index(drop=True)      
        
        title = self.country_selector
        
        fuel_list = _df.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = _df.hvplot.bar(
            x='trun',
            y='value',
            by='scenario',
            #stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)
        return plot    
    
    def plot_fuel_grid2(self):    

        country = self.country_selector
        _df  = self.get_data_fuel(country)
        ##mask = (_df['scenario']==self.scenario_selector)               # come back to this
        ##data = _df.loc[mask].reset_index(drop=True)      
        
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        title = self.country_selector
        
        fuel_list = df_pivot.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        # workaround for no stacked bar chart by scenario
        # does not stack though
        plot = df_pivot.hvplot.bar(
            x='trun',
            y='NG',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90) * df_pivot.hvplot.bar(
            x='trun',
            y='Arablight',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90) * df_pivot.hvplot.bar(
            x='trun',
            y='HFO',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)* df_pivot.hvplot.bar(
            x='trun',
            y='diesel',
            by='scenario',
            stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)
        return plot
    
    def plot_fuel_grid3(self):    

        country = self.country_selector
        _df  = self.get_data_fuel(country)
        df_pivot = _df.pivot_table(index=['c','trun','scenario'],columns='f',values='value',fill_value=0)
        df_pivot = df_pivot.reset_index(drop=False)
        
        title = self.country_selector
        
        fuel_list = df_pivot.drop(['c','trun','scenario'],axis=1).columns.values.tolist()

        plot = df_pivot.hvplot.bar(
            x='trun',
            y=fuel_list,
            by='scenario',
            #stacked=True,
            subplots=True,
            value_label='TBTU',
            title=title,
            alpha=0.4,
            width=300,
            legend=self.show_legend,
            shared_axes=True,
            rot=90)
        return plot 

Scatter plot works as expected:

In [ ]:
C = EMISSIONS()
C.country_selector = "Qatar"
#C.scenario_selector = "D"
C.plot_emissions_grid()

Following the same logic, trying a stacked bar plot:

In [ ]:
D = FUEL()
D.country_selector = "Saudi Arabia"
D.plot_fuel_grid()

I can manually overlay multiple bar plots, but not stacked:

In [ ]:
E = FUEL()
E.country_selector = "Saudi Arabia"
E.plot_fuel_grid2()

And one other way, using pivot. Note that only the first element of the y list is plotted:

In [ ]:
F = FUEL()
F.country_selector = "Saudi Arabia"
F.plot_fuel_grid3()